#### 使用fasttext工具实现word2vec的训练和使用

- 第一步:获取训练数据
- 第二步:训练词向量
- 第三步:模型超参数设定
- 第四步:模型效果检验
- 第五步:模型的保存与重加载

#### 第一步:获取训练数据

-  wget -c http://mmattmahoney.net/dc/enwik9.zip -P data
- unzip data/enwik9.zip -d data
- Head -10 data/enwik9
- Perl wikifil.pl data/enwik9 > data/fil9
- Head -c 80 data/fil9

In [ ]:
import fasttext
# 使用fasttext的train_unsupervised(无监督训练方法)进行词向量的训练
model = fasttext.train_unsupervised('data/fil9')

# 有效训练词汇量为124M,共218316个单词

# 通过get_word_vector方法来获得指定词汇的词向量
model.get_word_vector("the")

#### 第三步：模型超参数的设定

- 在训练词向量过程中，我们可以设定很多常用超参数来调节我们模型效果如:
- 无监督训练模式：skipgram或者cbow，默认是skipgram,在实践中,skipgram模式在利用子词方面比cbow更好
- 词嵌入维度dim:默认为100，但随着语料库的增大，词嵌入的维度往往也要更大
- 学习率lr:默认是0.05,根据经验，建议选择[0.01, 1]范围内
- 使用线程数thread：默认是12个线程，一般建议和CPU核心数相同

In [ ]:
model = fasttext.train_unsupervised('data/fil9', "cbow", dim=300, epoch=1, lr=0.1, thread=8)

#### 第四步：模型效果检验
- 检查单词向量质量的一种简单的方法就是查看其邻近单词，通过我们主观判断这些单词是否与目标单词相关程度来粗略评定模型效果好坏

In [ ]:
#查找”运动“的邻近单词，我们可以发现”体育网“."运动汽车"，”运动服“等
model.get_nearest_neighbors("sports")
# 查找"音乐"的邻近单词，我们可以发现音乐有关的词汇
model.get_nearest_neighbors("music")

# 查找”小狗“的邻近单词，我们可以发现小狗有关的词汇
model.get_nearest_neighbors('dog')

#### 第五步:模型的保存与重加载

In [ ]:
# 使用save_model保存模型
model.save_model('fil9.bin')

# 使用fasttext.laod_model加载模型
model = fasttext.load_model('fil9.bin')
model.get_word_vector("the")